In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import json 
import matplotlib
import matplotlib.pyplot as plt


from sklearn.metrics import jaccard_score
from scipy.spatial.distance import directed_hausdorff

import cv2
import tifffile
import seaborn as sns
from itertools import chain
from PIL import Image
from PIL import ImageDraw

import skimage
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#Read Metrics file for the team
BASE_PATH = os.getcwd()+'/'
INPUT_PATH = BASE_PATH
print(INPUT_PATH)

team = "team_3"
df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')
df_metrics.head()

In [3]:
df_masks = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/masks.csv')
df_masks.head()

,id,rle_truth,filename,tissue_name,rle_pred
0,3417,58904 2 60227 3 61550 4 62873 5 64196 6 65519 ...,D116-RML-BPS-2-2_0_0_22_lung,lung,324 171 1647 171 2970 171 4293 172 5616 173 69...
1,9077,592 15 1913 17 3235 19 4557 20 5879 22 7200 24...,D071-RLL-17B5-2_1323_1323_27_lung,lung,511 100 815 276 1834 102 2138 275 3157 103 346...
2,474,126388 11 127709 15 129031 18 130348 26 131669...,D356-RLL-12A3-2_0_0_16_lung,lung,14052 3 15374 6 16697 6 18020 7 19343 8 20665 ...
3,22423,34325 4 35647 6 36969 8 38291 10 39613 12 4093...,D231-RLL-13B4-2_0_0_32_lung,lung,830 250 1112 76 2153 250 2434 76 3476 250 3757...
4,16724,32875 3 34197 4 35518 6 36839 8 38161 10 39483...,D239-RLL-23C4-2_0_0_45_lung,lung,784 18 1031 17 2109 16 2354 18 3433 15 3677 19...


In [4]:
df_metadata = pd.read_csv(INPUT_PATH+'dataset_split_metadata/all_metadata_for_publication.csv')
df_metadata.head()

,tissue_name,antibody_id,patient_id,age,sex,tmaslide_id,data_type,filename,image_dims,min_area,max_area,mean_area,all_areas,tissue_thickness,pixel_size,rle,Usage
0,lung,41507.0,2268.0,49.0,Female,20136118.0,public,41507_88065_A_2_4_lung,"(3000, 3000, 3)",3771.5,26167.0,9355.00,"[6369.5, 26167.0, 3771.5, 8846.5, 6833.0, 6760...",4.0,0.4,2170394 3 2173394 6 2176394 7 2179395 7 218239...,public_train
1,lung,44657.0,2268.0,49.0,Female,20160434.0,public,44657_99514_A_2_4_lung,"(3000, 3000, 3)",6367.5,21650.5,11899.36,"[21650.5, 6367.5, 14240.0, 8710.0, 14189.5, 10...",4.0,0.4,1996630 8 1999626 14 2002624 16 2005622 19 200...,public_train
2,lung,68695.0,4840.0,43.0,Female,20361528.0,public,68695_162533_A_1_4_lung,"(3000, 3000, 3)",33705.5,33705.5,33705.50,[33705.5],4.0,0.4,4274535 30 4277533 34 4280531 37 4283529 39 42...,public_train
3,lung,35856.0,2208.0,67.0,Female,20094887.0,public,35856_71907_A_1_4_lung,"(3000, 3000, 3)",16816.5,46831.5,33955.00,"[29425.0, 44657.5, 16816.5, 23539.5, 42460.0, ...",4.0,0.4,3674181 11 3677180 14 3680179 18 3683177 23 36...,public_train
4,lung,54950.0,2208.0,67.0,Female,20275557.0,public,54950_136422_A_1_4_lung,"(3000, 3000, 3)",25690.5,25690.5,25690.50,[25690.5],4.0,0.4,4278843 10 4281842 13 4284842 15 4287841 20 42...,public_train


In [5]:
# Utility Methods

# Method to convert RLE to binary image mask
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m
   
    return img.reshape(shape).T

# # Method to visualize images and masks
# def show_image(filename,img_id, rle_truth, rle_pred, img_shape, organ, case, index):
#     fig, ax = plt.subplots(1,5,figsize=(20,40))

#     image = plt.imread(INPUT_PATH+f'published_data/data/data/images/{organ}/{filename}.tif')
#     mask_pred = enc2mask(rle_pred,shape=(img_shape[0],img_shape[1]))
#     mask_truth = enc2mask(rle_truth,shape=(img_shape[0],img_shape[1]))
#     mask_diff = mask_truth - mask_pred #np.abs(mask_truth - mask_pred)
#     # 1 [FN, not predicted]
#     mask_diff_fn = mask_diff == 1
#     mask_diff_fn = mask_diff_fn.astype(np.uint8)
#     # -1 (255) [FP, incorrectly predicted]
#     mask_diff_fp = mask_diff > 1
#     mask_diff_fp = mask_diff_fp.astype(np.uint8)
   
#     # diff_cmap = matplotlib.colors.ListedColormap(['blue', 'black', 'red'])
#     ax[0].imshow(image)
#     ax[0].set_title(f'file id: {img_id}')
#     ax[0].axis("off")

#     ax[1].imshow(mask_truth,alpha=0.7,cmap='gray')
#     #ax[1].imshow(mask_pred,alpha=0.3,cmap='Reds')
#     ax[1].set_title("Ground Truth")
#     ax[1].axis("off")

#     ax[2].imshow(mask_pred,alpha=0.7,cmap='gray')
#     #ax[2].imshow(mask_pred,alpha=0.3,cmap='Reds')
#     ax[2].set_title("Prediction")
#     ax[2].axis("off")

#     ax[3].imshow(mask_diff_fn,alpha=0.5,cmap='Reds')
#     ax[3].imshow(mask_diff_fp,alpha=0.5,cmap='Blues')
#     #ax[3].imshow(mask_pred,alpha=0.3,cmap='Reds')
#     ax[3].set_title("Mask Difference (FN: Red, FP: Blue)")
#     ax[3].axis("off")

#     ax[4].imshow(image)
#     ax[4].imshow(mask_pred,alpha=0.5,cmap='gray')
#     ax[4].set_title("Prediction Overlay")
#     ax[4].axis("off")

#     plt.savefig(INPUT_PATH+f'winning-submissions/{team}/viz/viz_{organ}_{case}_{index}.png', bbox_inches='tight')
#     plt.close(fig)
#     #plt.show()
#     print('-'*50)

# Method to visualize cases from get_n_cases(). 
def visualize_cases(df, case):
    fig, ax = plt.subplots(5,5,figsize=(15,15))
    i = 0

    for index, row in df.iterrows():
        organ = row['tissue_name']
        filename = row['filename']
        img_id = row['id']

        mask_row = df_masks[df_masks['id'] == img_id]
        rle_pred = mask_row['rle_pred']
        rle_truth = mask_row['rle_truth']
        shape = df_metadata[df_metadata['filename'] == filename]['image_dims'].iloc[0]
        shape = shape.strip().strip('(').strip(')').split(',')
        img_shape = [int(shape[0]), int(shape[1])]
        
        # show_image(filename, img_id, rle_truth, rle_pred, img_shape, organ, case, index)
        image = plt.imread(INPUT_PATH+f'published_data/data/data/images/{organ}/{filename}.tif')
        mask_pred = enc2mask(rle_pred,shape=(img_shape[0],img_shape[1]))
        mask_truth = enc2mask(rle_truth,shape=(img_shape[0],img_shape[1]))
        mask_diff = mask_truth - mask_pred #np.abs(mask_truth - mask_pred)
        # 1 [FN, not predicted]
        mask_diff_fn = mask_diff == 1
        mask_diff_fn = mask_diff_fn.astype(np.uint8)
        # -1 (255) [FP, incorrectly predicted]
        mask_diff_fp = mask_diff > 1
        mask_diff_fp = mask_diff_fp.astype(np.uint8)
    
        # diff_cmap = matplotlib.colors.ListedColormap(['blue', 'black', 'red'])
        if i == 0:
            ax[i][0].imshow(image)
            ax[i][0].set_title(f'Image')
            ax[i][0].axis("off")

            ax[i][1].imshow(mask_truth,alpha=0.7,cmap='gray')
            #ax[i][1].imshow(mask_pred,alpha=0.3,cmap='Reds')
            ax[i][1].set_title("Ground Truth")
            ax[i][1].axis("off")

            ax[i][2].imshow(mask_pred,alpha=0.7,cmap='gray')
            #ax[i][2].imshow(mask_pred,alpha=0.3,cmap='Reds')
            ax[i][2].set_title("Prediction")
            ax[i][2].axis("off")

            ax[i][3].imshow(mask_diff_fn,alpha=0.5,cmap='Reds')
            ax[i][3].imshow(mask_diff_fp,alpha=0.5,cmap='Blues')
            #ax[i][3].imshow(mask_pred,alpha=0.3,cmap='Reds')
            ax[i][3].set_title("Mask Difference (FN: Red, FP: Blue)")
            ax[i][3].axis("off")

            ax[i][4].imshow(image)
            ax[i][4].imshow(mask_pred,alpha=0.5,cmap='gray')
            ax[i][4].set_title("Prediction Overlay")
            ax[i][4].axis("off")
        else:
            ax[i][0].imshow(image)
            # ax[i][0].set_title(f'file id: {img_id}')
            ax[i][0].axis("off")

            ax[i][1].imshow(mask_truth,alpha=0.7,cmap='gray')
            #ax[i][1].imshow(mask_pred,alpha=0.3,cmap='Reds')
            # ax[i][1].set_title("Ground Truth")
            ax[i][1].axis("off")

            ax[i][2].imshow(mask_pred,alpha=0.7,cmap='gray')
            #ax[i][2].imshow(mask_pred,alpha=0.3,cmap='Reds')
            # ax[i][2].set_title("Prediction")
            ax[i][2].axis("off")

            ax[i][3].imshow(mask_diff_fn,alpha=0.5,cmap='Reds')
            ax[i][3].imshow(mask_diff_fp,alpha=0.5,cmap='Blues')
            #ax[i][3].imshow(mask_pred,alpha=0.3,cmap='Reds')
            # ax[i][3].set_title("Mask Difference (FN: Red, FP: Blue)")
            ax[i][3].axis("off")

            ax[i][4].imshow(image)
            ax[i][4].imshow(mask_pred,alpha=0.5,cmap='gray')
            # ax[i][4].set_title("Prediction Overlay")
            ax[i][4].axis("off")

        i += 1
        #break
    plt.tight_layout()
    plt.savefig(INPUT_PATH+f'winning-submissions/{team}/viz/viz_{organ}_five_{case}.svg', bbox_inches='tight', dpi=600, format='svg')
    plt.savefig(INPUT_PATH+f'winning-submissions/{team}/viz/viz_{organ}_five_{case}.png', bbox_inches='tight', dpi=600, format='png')
    plt.close(fig)
    
    # plt.show()
    # print('-'*50)
    
# Method to get n best or worst cases for an organ
def get_n_cases(df, organ, metric, ascending, n=5):
    if metric == 'dice':
        m = 'dice_score'
    elif metric == "iou":
        m = 'iou_score'
    # Select kidney images
    selected_rows = df[df['tissue_name'] == organ]
    # sort the dataframe in decreasing order based on the 'dice_score' column
    sorted_df = selected_rows.sort_values(by=m, ascending=ascending)

    # get the top 5 rows of the sorted dataframe
    n_cases = sorted_df.head(n)

    #Save to csv
    if ascending == True:
        case = 'worst'
    elif ascending == False:
        case = 'best'
    n_cases.to_csv(INPUT_PATH+f'winning-submissions/{team}/{organ}_{metric}_{case}_{n}.csv',index=None)

    return n_cases

# Get best 5 and worst 5 cases per organ for Dice.

### Best 5

In [6]:
# kidney
case = 'dice_best'
top_5 = get_n_cases(df_metrics, 'kidney', 'dice', False, 5)
top_5 = top_5.reset_index(drop=True)
display(top_5)

visualize_cases(top_5, case)    

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,3102,0.983750,0.968019,17.464249,470.177059,VAN0005-RK-8-11-PAS_FFPE_patch_8000_20000_47_k...,kidney
1,2922,0.980558,0.961857,22.627417,22.600582,VAN0014-LK-207-11-PAS_FFPE_patch_4000_20000_40...,kidney
2,9637,0.978626,0.958147,10.770330,64.443231,VAN0011-RK-8-11-PAS_FFPE_patch_28000_10000_34_...,kidney
3,3667,0.977745,0.956459,4.242641,1258.798650,VAN0028-LK-7-11-PAS_FFPE_patch_24000_16000_79_...,kidney
4,8246,0.976465,0.954012,17.029386,33.123516,VAN0010-LK-157-11-PAS_FFPE_patch_10000_4000_11...,kidney


In [7]:
# largeintestine
case = 'dice_best'
top_5 = get_n_cases(df_metrics, 'largeintestine', 'dice', False, 5)
top_5 = top_5.reset_index(drop=True)
display(top_5)

visualize_cases(top_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,14970,0.952692,0.909658,518.541223,507.941425,0_A002-C-104_cns_patch_1_0_largeintestine,largeintestine
1,32459,0.937706,0.882719,198.957282,1739.052518,1_EP091_cns_patch_0_0_largeintestine,largeintestine
2,17884,0.929774,0.868765,90.824006,2159.948633,JP_Desc_3_patch_0_0_largeintestine,largeintestine
3,11839,0.926383,0.862862,190.065778,615.266675,EP061_patch_0_0_largeintestine,largeintestine
4,13159,0.919662,0.851272,272.470182,271.137421,0_A002-C-211_cns_patch_2_0_largeintestine,largeintestine


In [8]:
# spleen
case = 'dice_best'
top_5 = get_n_cases(df_metrics, 'spleen', 'dice', False, 5)
top_5 = top_5.reset_index(drop=True)
display(top_5)

visualize_cases(top_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,14967,0.944031,0.893995,158.672619,580.023757,179750_0_0_45_spleen,spleen
1,14069,0.939920,0.886650,64.815122,489.700127,163783_patch_0_0_spleen,spleen
2,6365,0.936331,0.880284,695.149624,662.653227,179750_0_0_42_spleen,spleen
3,7572,0.935837,0.879412,403.019851,385.368858,179757_0_0_29_spleen,spleen
4,29615,0.931044,0.870984,750.501166,713.676108,179759_0_0_66_spleen,spleen


In [9]:
# lung
case = 'dice_best'
top_5 = get_n_cases(df_metrics, 'lung', 'dice', False, 5)
top_5 = top_5.reset_index(drop=True)
display(top_5)

visualize_cases(top_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,15723,0.910934,0.836435,125.239770,129.194338,D231-RLL-11A5-2_0_0_64_lung,lung
1,5336,0.904974,0.826441,464.698827,454.858008,D071-RLL-17B5-2_2646_1323_29_lung,lung
2,2324,0.884051,0.792196,260.863949,252.691583,D265-LLL-7A7-2_0_0_35_lung,lung
3,21589,0.882531,0.789758,122.576507,221.131267,D292-RLL-8A4-2_0_0_52_lung,lung
4,11625,0.871795,0.772728,104.201727,101.962331,D071-RLL-17C2-32_0_1323_8_lung,lung


In [10]:
# prostate
case = 'dice_best'
top_5 = get_n_cases(df_metrics, 'prostate', 'dice', False, 5)
top_5 = top_5.reset_index(drop=True)
display(top_5)

visualize_cases(top_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,8733,0.958344,0.920019,28.792360,28.476352,D080-Sag1-LUT-1_0_0_5_prostate,prostate
1,8415,0.952589,0.909469,5.099020,9.802333,D070-Sag2-LUT-7_0_0_22_prostate,prostate
2,13145,0.951950,0.908306,15.033296,27.789139,D096-Sag3-LUT-1_0_0_32_prostate,prostate
3,20703,0.947151,0.899608,8.062258,73.577784,D077-Sag2-LUT-5_0_0_52_prostate,prostate
4,22077,0.946393,0.898242,26.925824,25.970045,D092-Sag3-LUT-1_0_0_63_prostate,prostate


### Worst 5

In [11]:
# kidney
case = 'dice_worst'
worst_5 = get_n_cases(df_metrics, 'kidney', 'dice', True, 5)
worst_5 = worst_5.reset_index(drop=True)
display(worst_5)

visualize_cases(worst_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,23540,0.821487,0.697054,824.602935,784.767421,VAN0013-LK-208-11-PAS_FFPE_patch_4000_10000_18...,kidney
1,9717,0.894441,0.809040,8.000000,498.407543,VAN0009-LK-106-11-PAS_FFPE_patch_4000_8000_36_...,kidney
2,22616,0.895281,0.810415,5.000000,679.266778,VAN0012-RK-108-11-PAS_FFPE_patch_28000_22000_7...,kidney
3,29717,0.907472,0.830617,12.727922,644.918524,VAN0014-LK-206-11-PAS_FFPE_patch_24000_12000_6...,kidney
4,4252,0.916375,0.845656,617.767756,1177.218397,VAN0035-LK-8-11-PAS_FFPE_patch_22000_16000_85_...,kidney


In [12]:
# largeintestine
case = 'dice_worst'
worst_5 = get_n_cases(df_metrics, 'largeintestine', 'dice', True, 5)
worst_5 = worst_5.reset_index(drop=True)
display(worst_5)

visualize_cases(worst_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,9276,0.498918,0.332372,224.555116,1196.346568,2_A001-C-224_cns_patch_0_0_largeintestine,largeintestine
1,10655,0.727628,0.571868,284.200633,1324.400905,A001-C-023_patch_0_0_largeintestine,largeintestine
2,3701,0.760825,0.613977,700.163552,1063.835370,A002-C-119_patch_1_0_largeintestine,largeintestine
3,16839,0.795414,0.660321,300.820545,1090.738071,3_A001-C-224_cns_patch_0_0_largeintestine,largeintestine
4,2499,0.813709,0.685927,49.040799,802.450193,JP025_patch_1_0_largeintestine,largeintestine


In [13]:
# spleen
case = 'dice_worst'
worst_5 = get_n_cases(df_metrics, 'spleen', 'dice', True, 5)
worst_5 = worst_5.reset_index(drop=True)
display(worst_5)

visualize_cases(worst_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,21524,0.571734,0.400299,40.000000,325.208095,179753_0_0_83_spleen,spleen
1,27182,0.579514,0.407969,517.919878,575.236728,178800_0_0_51_spleen,spleen
2,29651,0.689382,0.525997,699.927139,671.281569,178800_0_0_54_spleen,spleen
3,18472,0.699125,0.537427,233.452351,233.160089,179752_0_0_95_spleen,spleen
4,4898,0.718564,0.560749,637.251913,971.009071,179752_0_0_98_spleen,spleen


In [14]:
# lung
case = 'dice_worst'
worst_5 = get_n_cases(df_metrics, 'lung', 'dice', True, 5)
worst_5 = worst_5.reset_index(drop=True)
display(worst_5)

visualize_cases(worst_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,22517,0.193324,0.107006,276.371127,636.114967,D116-RLL-12A4-2_1323_0_4_lung,lung
1,2853,0.437423,0.279937,18.027756,856.763342,D291-RUL-12A2-2_0_0_10_lung,lung
2,6329,0.496416,0.330155,694.348616,664.506634,D294-RLL-15A4-2_0_0_11_lung,lung
3,10029,0.515601,0.347347,459.252654,446.668123,D356-RLL-12A3-2_0_0_15_lung,lung
4,3789,0.539144,0.369060,632.949445,625.143533,D122-RLL-25C1-2_1323_0_3_lung,lung


In [15]:
# prostate
case = 'dice_worst'
worst_5 = get_n_cases(df_metrics, 'prostate', 'dice', True, 5)
worst_5 = worst_5.reset_index(drop=True)
display(worst_5)

visualize_cases(worst_5, case)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,24079,0.056272,0.028951,146.492321,139.422656,D070-Sag2-LUT-7_0_0_26_prostate,prostate
1,23140,0.326328,0.194977,121.082616,115.228486,D070-Sag2-LUT-7_0_0_30_prostate,prostate
2,31209,0.462085,0.300462,123.943536,117.969966,D096-Sag3-LUT-1_0_0_37_prostate,prostate
3,18408,0.510354,0.342601,71.309186,67.885148,D092-Sag3-LUT-1_0_0_67_prostate,prostate
4,10420,0.552433,0.381629,81.633327,78.451660,D093-Sag1-LUT-1_0_0_17_prostate,prostate


# Get best 5 and worst 5 cases per organ for IOU.

### Best 5

In [ ]:
# # kidney
# case = 'iou_best'
# top_5 = get_n_cases(df_metrics, 'kidney', 'iou', False, 5)
# top_5 = top_5.reset_index(drop=True)
# display(top_5)

# visualize_cases(top_5, case)

In [ ]:
# # largeintestine
# case = 'iou_best'
# top_5 = get_n_cases(df_metrics, 'largeintestine', 'iou', False, 5)
# top_5 = top_5.reset_index(drop=True)
# display(top_5)

# visualize_cases(top_5, case)

In [ ]:
# # spleen
# case = 'iou_best'
# top_5 = get_n_cases(df_metrics, 'spleen', 'iou', False, 5)
# top_5 = top_5.reset_index(drop=True)
# display(top_5)

# visualize_cases(top_5, case)

In [ ]:
# # lung
# case = 'iou_best'
# top_5 = get_n_cases(df_metrics, 'lung', 'iou', False, 5)
# top_5 = top_5.reset_index(drop=True)
# display(top_5)

# visualize_cases(top_5, case)

In [ ]:
# # prostate
# case = 'iou_best'
# top_5 = get_n_cases(df_metrics, 'prostate', 'iou', False, 5)
# top_5 = top_5.reset_index(drop=True)
# display(top_5)

# visualize_cases(top_5, case)

### Worst 5

In [ ]:
# # kidney
# case = 'iou_worst'
# worst_5 = get_n_cases(df_metrics, 'kidney', 'iou', True, 5)
# worst_5 = worst_5.reset_index(drop=True)
# display(worst_5)

# visualize_cases(worst_5, case)

In [ ]:
# # largeintestine
# case = 'iou_worst'
# worst_5 = get_n_cases(df_metrics, 'largeintestine', 'iou', True, 5)
# worst_5 = worst_5.reset_index(drop=True)
# display(worst_5)

# visualize_cases(worst_5, case)

In [ ]:
# # spleen
# case = 'iou_worst'
# worst_5 = get_n_cases(df_metrics, 'spleen', 'iou', True, 5)
# worst_5 = worst_5.reset_index(drop=True)
# display(worst_5)

# visualize_cases(worst_5, case)

In [ ]:
# # lung
# case = 'iou_worst'
# worst_5 = get_n_cases(df_metrics, 'lung', 'iou', True, 5)
# worst_5 = worst_5.reset_index(drop=True)
# display(worst_5)

# visualize_cases(worst_5, case)

In [ ]:
# # prostate
# case = 'iou_worst'
# worst_5 = get_n_cases(df_metrics, 'prostate', 'iou', True, 5)
# worst_5 = worst_5.reset_index(drop=True)
# display(worst_5)

# visualize_cases(worst_5, case)

In [ ]:
# def enc2mask(encs, shape):
#     img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
#     for m, enc in enumerate(encs):
#         if isinstance(enc, float) and np.isnan(enc):
#             continue
#         enc_split = enc.split()
#         for i in range(len(enc_split) // 2):
#             start = int(enc_split[2 * i]) - 1
#             length = int(enc_split[2 * i + 1])
#             img[start: start + length] = 1 + m
#     return img.reshape(shape).T

In [ ]:
# def show_image(filename, rle_truth, rle_pred, img_shape, organ, case, index):
#     fig, ax = plt.subplots(1,4,figsize=(20,40))

#     image = plt.imread(INPUT_PATH+f'published_data/data/data/images/{organ}/{filename}.tif')
#     mask_pred = enc2mask(rle_pred,shape=(img_shape[0],img_shape[1]))
#     mask_truth = enc2mask(rle_truth,shape=(img_shape[0],img_shape[1]))
#     mask_diff = mask_truth - mask_pred
#     #display(pd.DataFrame(df_organ.loc[idx,['id','organ','age','sex']]).T)
#     ax[0].imshow(image)
#     ax[0].set_title("Image")
#     ax[0].axis("off")
#     ax[1].imshow(mask_pred,alpha=0.7,cmap='gray')
#     #ax[1].imshow(mask_pred,alpha=0.3,cmap='Reds')
#     ax[1].set_title("Predicted Mask")
#     ax[1].axis("off")
#     ax[2].imshow(mask_diff,alpha=0.7,cmap='plasma')
#     #ax[1].imshow(mask_pred,alpha=0.3,cmap='Reds')
#     ax[2].set_title("Mask Difference")
#     ax[2].axis("off")
#     ax[3].imshow(image)
#     ax[3].imshow(mask_pred,alpha=0.5,cmap='gray')
#     ax[3].set_title("Prediction Overlay")
#     ax[3].axis("off")
#     plt.savefig(INPUT_PATH+f'winning-submissions/{team}/viz/viz_{case}_{index}.png', bbox_inches='tight')
#     plt.show()
#     print('-'*50)

In [ ]:
# def visualize_cases(df, case):
#     for index, row in df.iterrows():
#         organ = row['tissue_name']
#         filename = row['filename']
#         img_id = row['id']

#         mask_row = df_masks[df_masks['id'] == img_id]
#         rle_pred = mask_row['rle_pred']
#         rle_truth = mask_row['rle_truth']
#         shape = df_metadata[df_metadata['filename'] == filename]['image_dims'].iloc[0]
#         shape = shape.strip().strip('(').strip(')').split(',')
#         img_shape = [int(shape[0]), int(shape[1])]
        
#         show_image(filename, rle_truth, rle_pred, img_shape, organ, case, index)
#         #break
    

In [ ]:
# # for best 5 kidney dice
# case = 'dice_best'
# top_5 = get_n_cases(df_metrics, 'kidney', 'dice', False, 5)
# top_5 = top_5.reset_index(drop=True)
# display(top_5)

# visualize_cases(top_5, case)